<a href="https://colab.research.google.com/github/YouCanKeepSilence/ml_labs/blob/master/ML_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт данных

In [0]:
import numpy as np
import pandas as pd

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Загружаем данные из .csv

In [0]:
names_array = ['gender', 'age', 'first_label', 'second_label', 'third_label', *['c_{}'.format(i + 1) for i in range(60000)]]
url = '/content/drive/My Drive/ML/half.csv'
data = pd.read_csv(url, names=names_array)

# Обработка данных

Удаляем ненужные стобцы (по причине малого количества данных в них)

In [0]:
data = data.drop(['second_label', 'third_label'], axis=1)

In [13]:
#new_data = data # to skip augmentation because it'll break PCA method
#new_batch = 5000
#sensors_count = 12
data

,gender,age,first_label,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,...,c_59961,c_59962,c_59963,c_59964,c_59965,c_59966,c_59967,c_59968,c_59969,c_59970,c_59971,c_59972,c_59973,c_59974,c_59975,c_59976,c_59977,c_59978,c_59979,c_59980,c_59981,c_59982,c_59983,c_59984,c_59985,c_59986,c_59987,c_59988,c_59989,c_59990,c_59991,c_59992,c_59993,c_59994,c_59995,c_59996,c_59997,c_59998,c_59999,c_60000
0,Female,85,3,0.003242,0.012132,0.022637,0.010953,-0.024891,-0.032752,-0.031432,-0.042890,-0.048659,-0.037955,-0.026816,-0.017216,-0.014815,-0.024370,-0.027953,-0.037105,-0.046662,-0.052641,-0.054785,-0.054178,-0.034856,-0.023726,-0.036748,-0.031842,-0.021913,-0.038332,-0.045422,-0.044497,-0.042550,-0.038772,-0.043089,-0.055117,-0.062105,-0.062300,-0.062635,-0.068795,-0.070163,...,-0.026842,-0.028914,-0.028548,-0.029272,-0.031896,-0.036031,-0.039794,-0.033593,-0.028113,-0.031301,-0.041995,-0.045999,-0.043968,-0.039291,-0.030942,-0.031698,-0.030754,-0.033479,-0.033446,-0.036413,-0.038551,-0.035920,-0.036477,-0.036518,-0.041184,-0.039042,-0.033992,-0.033940,-0.031528,-0.026267,-0.028246,-0.028318,-0.025569,-0.021279,-0.018593,-0.032023,-0.046202,-0.042003,-0.034466,-0.039000
1,Male,79,9,0.012054,0.017017,0.015951,0.020941,0.020960,0.021158,0.021051,0.021062,0.021122,0.021278,0.026533,0.026235,0.016037,0.013366,0.013222,0.008356,0.005556,0.007938,0.008593,0.008398,0.008547,0.008369,0.008677,0.006235,0.000651,-0.001857,-0.001712,-0.007047,-0.009004,-0.012014,-0.019256,-0.019027,-0.019518,-0.018919,-0.020128,-0.021534,-0.024676,...,-0.934049,-0.949376,-0.962051,-0.963686,-0.961742,-0.952030,-0.939060,-0.934415,-0.936366,-0.939326,-0.932011,-0.926725,-0.928331,-0.929013,-0.921389,-0.906309,-0.900787,-0.901273,-0.895911,-0.893740,-0.891627,-0.885869,-0.881577,-0.873489,-0.866113,-0.865862,-0.859139,-0.852907,-0.843380,-0.833925,-0.833698,-0.836701,-0.827418,-0.827810,-0.816231,-0.819793,-0.797862,-0.819597,-0.771195,-0.845490
2,Male,66,3,-0.009508,-0.008508,-0.009508,-0.008508,-0.007508,-0.006508,-0.006508,-0.006508,-0.005508,-0.006508,-0.005508,-0.004508,-0.003508,-0.004508,-0.004508,-0.004508,-0.005508,-0.004508,-0.003508,-0.004508,-0.003508,-0.002508,-0.000508,-0.001508,-0.000508,-0.000508,-0.001508,-0.001508,-0.001508,-0.000508,0.000492,0.002492,0.005492,0.006492,0.007492,0.008492,0.008492,...,-0.051053,-0.052053,-0.051053,-0.049053,-0.047053,-0.044053,-0.040053,-0.037053,-0.036053,-0.034053,-0.034053,-0.034053,-0.033053,-0.031053,-0.028053,-0.025053,-0.022053,-0.019053,-0.017053,-0.017053,-0.016053,-0.016053,-0.014053,-0.013053,-0.010053,-0.007053,-0.003053,0.001947,0.004947,0.007947,0.009947,0.011947,0.012947,0.015947,0.018947,0.023947,0.027947,0.030947,0.034947,0.037947
3,Male,93,2,-0.002271,0.004177,-0.002077,-0.005875,-0.004685,-0.008117,-0.019722,-0.036185,-0.037616,-0.025323,-0.023199,-0.035744,-0.043484,-0.030469,-0.022611,-0.010540,0.004805,0.020337,0.029668,0.017382,0.004737,0.002755,0.014075,0.015677,0.002024,-0.002544,0.000068,0.000271,0.011654,0.022805,0.028631,0.029834,0.030584,0.035327,0.051485,0.059849,0.036704,...,-0.117502,-0.208445,-0.254530,-0.270954,-0.254278,-0.223832,-0.205055,-0.179093,-0.157585,-0.127692,-0.089728,-0.074150,-0.056856,-0.038932,-0.024184,-0.016817,-0.014031,-0.011765,-0.022771,-0.023586,-0.013769,-0.004949,-0.003625,-0.013567,-0.019471,-0.015372,-0.021787,-0.028885,-0.028997,-0.021737,-0.013160,-0.006182,-0.003825,-0.002177,-0.001466,-0.006676,-0.011236,-0.016540,-0.016289,-0.022542
4,Male,65,2,-0.030600,-0.027425,-0.013923,-0.023743,-0.024572,-0.036078,-0.034740,-0.013670,-0.009571,-0.022441,-0.033034,-0.032663,-0.018181,-0.017317,-0.030576,-0.032829,-0.010305,0.005287,0.006696,-0.002352,-0.005559,-0.005361,-0.008254,0.002175,0.007531,0.002662,0.000376,-0.007697,-0.016977,-0.009894,-0.000388,-0.012127,-0.007840,-0.002470,-0.005233,0.004789,-0.007276,...,1.016910,0.710028,0.4

## Агументация данных

In [0]:
# batch_size = 1000
new_batch = 250
measurments_for_sensors = 5000
sensors_count = 12

def augmentate():
  measurment_names = data.columns[3:]
  new_data_inner = pd.DataFrame()
  # Пользуемся тем, что имеем 5к измерений для каждого из 12 датчиков. Соответственно нарежем по 250 с каждого из 12 датчиков
  for i in range(0, measurments_for_sensors, new_batch):
      current_measurment_names = {f'c_{k + j * measurments_for_sensors + 1}': f'c_{(new_batch * j + k % new_batch) % (new_batch * sensors_count)  + 1}' for j in range(sensors_count) for k in range(i, i + new_batch)}
      mapped_piece = data[['age', 'gender', 'first_label', *[f'c_{k + j * measurments_for_sensors + 1}'  for j in range(sensors_count) for k in range(i, i + new_batch)]]].rename(columns=current_measurment_names)
      new_data_inner = pd.concat([new_data_inner, mapped_piece])
  return new_data_inner

# don't call augmentate...
new_data = data

Таким образом получаем из размерности 1250, 60003 -> 25000, 3003 увеличив количество записей в датасете в 20 раз



## Обработаем пропущенные значения

In [15]:
print(new_data['age'].unique())
print(new_data['gender'].unique())
print(new_data['first_label'].unique())
print(new_data.isnull().sum())

[85 79 66 93 65 55 82 87 51 61 90 77 70 60 62 59 84 71 45 48 75 49 72 78
 86 46 94 33 74 64 67 63 88 32 92 68 76 54 89 31 80 30 57 83 34 27 73 69
 52 44 43 47 39 56 81 53 50 16 29 37 58 24 22 40 41 11 26 28 96 35 91 25
 36 23 19 17 42 18 15  8 20 38 14 21]
['Female' 'Male']
[3 9 2 5 1 7 6 4 8]
gender         0
age            0
first_label    0
c_1            0
c_2            0
              ..
c_59996        0
c_59997        0
c_59998        0
c_59999        0
c_60000        0
Length: 60003, dtype: int64


Пропущенных значений нет.

## Обработаем категориальные признаки

In [18]:
categorial_columns = ['gender', 'first_label']
numerical_columns = ['age', *[f'c_{i + 1}' for i in range(new_batch * sensors_count)]]
print(categorial_columns)

['gender', 'first_label']


Проведем бинаризацию пола, label не трогаем, им займемся дальше

In [19]:
new_data.at[data['gender'] == 'Male', 'gender'] = 0
new_data.at[data['gender'] == 'Female', 'gender'] = 1
new_data['gender'].describe()

count    1250.000000
mean        0.392800
std         0.488568
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: gender, dtype: float64

In [21]:
categorial_columns.remove('gender')
numerical_columns.append('gender')

ValueError: ignored

## Произведем нормировку данных

In [22]:
numerical_data = new_data[numerical_columns]
numerical_data.describe()

,age,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,c_38,c_39,...,c_2962,c_2963,c_2964,c_2965,c_2966,c_2967,c_2968,c_2969,c_2970,c_2971,c_2972,c_2973,c_2974,c_2975,c_2976,c_2977,c_2978,c_2979,c_2980,c_2981,c_2982,c_2983,c_2984,c_2985,c_2986,c_2987,c_2988,c_2989,c_2990,c_2991,c_2992,c_2993,c_2994,c_2995,c_2996,c_2997,c_2998,c_2999,c_3000,gender
count,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,...,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000
mean,65.856000,-0.001139,-0.002105,-0.001186,-0.000699,0.000080,0.001231,0.001647,0.000844,-0.000507,-0.001404,-0.000979,0.000224,0.001310,0.001309,0.001362,0.001244,0.001325,0.000977,-0.000244,-0.001267,-0.001956,-0.002275,-0.002705,-0.003862,-0.003565,-0.002761,-0.002193,-0.001470,-0.000565,0.000766,0.001641,0.001913,0.002586,0.003234,0.003087,0.003314,0.003607,0.003085,0.002829,...,-0.003779,-0.003539,-0.003422,-0.003116,-0.002223,-0.001412,-0.001079,-0.000650,0.000196,0.001152,0.001612,0.001681,0.002155,0.002150,0.002135,0.002641,0.002507,0.002675,0.003116,0.003356,0.002281,0.001424,0.001440,0.001190,0.000496,-0.000281,-0.001168,-0.002173,-0.002865,-0.003451,-0.004338,-0.004762,-0.004489,-0.004623,-0.004062,-0.002394,-0.001705,-0.001390,-0.001172,0.392800
std,16.259704,0.109153,0.147960,0.142419,0.154217,0.156632,0.164304,0.167253,0.160772,0.148017,0.144894,0.151951,0.161172,0.163977,0.162418,0.158913,0.155349,0.152982,0.148850,0.139679,0.135292,0.136578,0.136331,0.134528,0.133113,0.132706,0.133541,0.135828,0.138968,0.139904,0.141987,0.145540,0.148112,0.150525,0.152528,0.154053,0.154859,0.156462,0.158158,0.158931,...,0.137687,0.138259,0.137725,0.136511,0.136332,0.137962,0.140577,0.142708,0.144332,0.145893,0.146820,0.147875,0.147909,0.147213,0.146543,0.146174,0.146101,0.145409,0.144659,0.144144,0.144328,0.145167,0.145816,0.145355,0.143266,0.140423,0.138445,0.136507,0.134620,0.134341,0.135831,0.137995,0.140785,0.143307,0.145355,0.147054,0.147757,0.148150,0.148407,0.488568
min,8.000000,-1.016408,-1.434000,-1.275380,-1.341125,-1.282850,-1.300322,-1.297810,-1.333237,-1.328404,-1.317127,-1.305979,-1.298437,-1.294517,-1.299913,-1.303422,-1.300979,-1.300096,-1.302507,-1.300547,-1.287951,-1.272765,-1.265217,-1.264992,-1.255821,-1.239406,-1.245607,-1.279264,-1.291142,-1.299759,-1.297960,-1.293952,-1.294623,-1.295910,-1.317300,-1.328641,-1.318360,-1.320130,-1.325222,-1.303114,...,-1.542082,-1.574492,-1.560768,-1.534844,-1.535215,-1.525890,-1.539868,-1.550668,-1.540830,-1.532384,-1.539832,-1.568638,-1.583085,-1.567808,-1.529055,-1.494042,-1.484855,-1.466341,-1.496133,-1.507820,-1.499618,-1.500326,-1.510475,-1.545599,-1.552249,-1.534214,-1.572994,-1.583210,-1.585905,-1.609331,-1.622633,-1.574256,-1.529685,-1.523265,-1.527289,-1.547521,-1.542461,-1.535634,-1.517420,0.000000
25%,58.000000,-0.033989,-0.048053,-0.042730,-0.042909,-0.041418,-0.042272,-0.041362,-0.043821,-0.043686,-0.043752,-0.046005,-0.042629,-0.042886,-0.042932,-0.041426,-0.041942,-0.042938,-0.042431,-0.043416,-0.042570,-0.042112,-0.043279,-0.041860,-0.044

In [0]:
mean = numerical_data.mean(axis = 0)
std = numerical_data.std(axis = 0)
numerical_data = (numerical_data - mean) / std
new_data[numerical_columns] = numerical_data

In [24]:
new_data.describe()

,gender,age,first_label,c_1,c_2,c_3,c_4,c_5,c_6,c_7,c_8,c_9,c_10,c_11,c_12,c_13,c_14,c_15,c_16,c_17,c_18,c_19,c_20,c_21,c_22,c_23,c_24,c_25,c_26,c_27,c_28,c_29,c_30,c_31,c_32,c_33,c_34,c_35,c_36,c_37,...,c_59961,c_59962,c_59963,c_59964,c_59965,c_59966,c_59967,c_59968,c_59969,c_59970,c_59971,c_59972,c_59973,c_59974,c_59975,c_59976,c_59977,c_59978,c_59979,c_59980,c_59981,c_59982,c_59983,c_59984,c_59985,c_59986,c_59987,c_59988,c_59989,c_59990,c_59991,c_59992,c_59993,c_59994,c_59995,c_59996,c_59997,c_59998,c_59999,c_60000
count,1.250000e+03,1.250000e+03,1250.000000,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,1.250000e+03,...,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000,1250.000000
mean,-4.112266e-17,3.527845e-16,4.007200,2.526868e-17,-7.416290e-18,-1.444400e-17,-3.981815e-17,-4.996004e-18,7.405188e-18,-2.821077e-17,1.900424e-17,-2.209344e-17,1.318945e-17,-2.238210e-17,3.417266e-17,-6.443734e-17,2.673417e-17,-3.217426e-17,1.145750e-17,3.317346e-17,2.051692e-17,-1.949552e-17,-7.727152e-18,2.877698e-17,5.417888e-18,-1.358913e-17,3.552714e-17,4.272138e-17,9.725554e-18,1.452172e-17,-8.970602e-18,-6.483702e-18,2.713385e-17,8.615331e-18,-5.506706e-18,9.303669e-18,-4.178879e-17,-2.637890e-17,2.664535e-18,2.793321e-17,...,-0.000564,0.000351,0.001139,0.002124,0.003319,0.003851,0.003792,0.004112,0.004451,0.004071,0.003836,0.003528,0.002257,0.000447,-0.001505,-0.003825,-0.005995,-0.007449,-0.008046,-0.008391,-0.008372,-0.008107,-0.007929,-0.007539,-0.006885,-0.006192,-0.005158,-0.003966,-0.002890,-0.002123,-0.001556,-0.001544,-0.002460,-0.004072,-0.005804,-0.008189,-0.010095,-0.012380,-0.012375,-0.014290
std,1.000000e+00,1.000000e+00,2.206863,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.289376,0.290498,0.293803,0.299285,0.304464,0.308960,0.312213,0.314095,0.315738,0.317939,0.320239,0.321139,0.319850,0.316424,0.310127,0.299973,0.289762,0.281493,0.273258,0.265423,0.259596,0.256717,0.255889,0.257820,0.261944,0.266477,0.273977,0.286926,0.302027,0.312504,0.316296,0.316760,0.314992,0.311292,0.299572,0.288294,0.273217,0.274725,0.260574,0.281653
min,-8.039815e-01,-3.558244e+00,1.000000,-9.301299e+00,-9.677606e+00,-8.946761e+00,-8.691844e+00,-8.190728e+00,-7.921619e+00,-7.769412e+00,-8.297991e+00,-8.971252e+00,-9.080596e+00,-8.588321e+00,-8.057595e+00,-7.902499e+00,-8.011567e+00,-8.210700e+00,-8.382546e+00,-8.507031e+00,-8.757037e+00,-9.309247e+00,-9.510396e+00,-9.304666e+00,-9.263769e+00,-9.383047e+00,-9.405265e+00,-9.312604e+00,-9.306829e+00,-9.402096e+00,-9.280372e+00,-9.286357e+00,-9.146765e+00,-8.901995e+00,-8.753763e+00,-8.626441e+00,-8.657630e+00,-8.644601e+00,-8.534698e+00,-8.460449e+00,...,-3.100451,-3.061930,-3.103308,-3.213216,-3.326783,-3.458441,-3.583804,-3.645128,-3.631912,

Нормализовали все поля кроме label т.к. он является выходным и его нет смысла нормализовывать.

# Подготовка данных для модели

In [25]:
from sklearn.model_selection import train_test_split
y = new_data['first_label'].values
X = new_data.drop(['first_label'], axis=1).values

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

n_train, _ = x_train.shape 
n_test,  _ = x_test.shape 

print('Train dataset size: {}'.format(n_train))
print('Test dataset size: {}'.format(n_test))

Train dataset size: 875
Test dataset size: 375


# Создание и обучение моделей

## KMeans

In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=len(new_data['first_label'].unique()), random_state=42).fit(X)

In [27]:
kmeans_cluster = kmeans.predict(X) + 1
print(np.unique(kmeans.labels_))
check = kmeans_cluster == y
np.where(check == True)[0].shape

[0 1 2 3 4 5 6 7 8]


(90,)

Т.к. разбиение подразумевает передачу количества классов, то количество классов совпадает, однако пересечение с реальным разбиением достаточно слабое. (90)

## DBSCAN

In [0]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN().fit(X)

In [29]:
np.unique(dbscan.labels_)

array([-1])

На агументированных данных с таким набором параметров разбиение происходит на 5 классов, в то время как изначальное разбиение произведено на 9 классов.
На неагументированных данных не обучается. (вернуло массив, содержащий только -1)

## Affinity Propagation


In [30]:
from sklearn.cluster import AffinityPropagation
affinity = AffinityPropagation().fit(X)

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/_affinity_propagation.py:226: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  "will not have any cluster centers.", ConvergenceWarning)


In [31]:
np.unique(affinity.labels_)

array([-1])

На аугментированных данных - не удалось проверить ибо вылезло за пределы по памяти (более 25 Гб оперативки).
На неаугментированных - не обучилось. (вернуло массив, содержащий только -1)

## Spectral Clustering


In [36]:
from sklearn.cluster import SpectralClustering
spectral = SpectralClustering(n_clusters=len(new_data['first_label'].unique()), random_state=42).fit_predict(X)

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [38]:
# spectral_cluster = spectral.predict(X) +
check_spectral = spectral + 1 == y
# check_spectral = spectral_cluster == y
np.where(check_spectral == True)[0].shape

(169,)

На неаугментированных данных разбиение имеет большее пересечение с оригинальным, чем KMeans. (169)
На аугментированных обучение продолжается очень долго (более двух часов) не дождался.

## Birch


In [0]:
from sklearn.cluster import Birch
birch = Birch(n_clusters=len(new_data['first_label'].unique())).fit(X)

In [39]:
birch_cluster = birch.predict(X) + 1
print(np.unique(birch.labels_))
check_birch = birch_cluster == y
np.where(check_birch == True)[0].shape

[0 1 2 3 4 5 6 7 8]


(169,)

На неагументированных данных имеет такое же количество пересечений с оригинальным разбиением как и Spectral Clustering. (169)

# Вывод
Были опробованы следующие методы кластеризации: KMeans, DBscan, Affinity Propagation, Spectral Clustering, Birch.

Подвердить или опровергнуть достоверность их разбиения непредставляется возможным. Однако, наибольшее пересечение с оригинальным разбиением дали Birch и Spectral Clustering